## 資料前處理

In [3]:
import os 
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model, load_model
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
from keras.layers import Input, Dense, Flatten
from keras.layers.convolutional import AveragePooling2D
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input

import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import os
import math

import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn #ignore annoying warning (from sklearn and seaborn)

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\

In [4]:
import tensorflow as tf  
from keras.backend.tensorflow_backend import set_session  
config = tf.ConfigProto()  
config.gpu_options.allow_growth = True  
sess = tf.Session(config=config)  
set_session(sess)

In [5]:
#取得trainData
#print(os.getcwd())
data_path = 'C:/Users/n000171823/Desktop/PQ42'
trainSet = pd.DataFrame([])
# images 的路徑
path_list = os.listdir(data_path + '/training/')
for path in path_list:
    fp = []
    file_list = os.listdir(data_path + '/training/' + path + '/')
    for f in file_list:
        fp.append(data_path + '/training/' + path + '/' + f)
    trainSet = pd.concat([trainSet, pd.DataFrame({'file_path':fp, 'class':path})], ignore_index=True)

print(len(trainSet))

if 'Thumbs' in trainSet['file_path']:
    print('T')
#     trainSet.drop()
print(trainSet['file_path'][:5])  

79559
0    C:/Users/n000171823/Desktop/KGD9分類瑕疵圖檔/trainin...
1    C:/Users/n000171823/Desktop/KGD9分類瑕疵圖檔/trainin...
2    C:/Users/n000171823/Desktop/KGD9分類瑕疵圖檔/trainin...
3    C:/Users/n000171823/Desktop/KGD9分類瑕疵圖檔/trainin...
4    C:/Users/n000171823/Desktop/KGD9分類瑕疵圖檔/trainin...
Name: file_path, dtype: object


In [4]:
#定義目標類別
classes = ['01刮傷','02模痕','03摺痕','04晶點','05膠化不良',
           '06黑點','07異物','08破孔','09結疤','10油污','11蚊蟲','12其他']

for i in range(len(classes)):
    trainSet['class'][trainSet['class'] == classes[i]] = i
trainSet.head()

# print(len(classes))

for i in range(0,len(classes)):
    print(i,classes[i])
print('=======================') 
print(trainSet['class'].value_counts())
print(len(trainSet))
trainSet.head()

0 01刮傷
1 02模痕
2 03摺痕
3 04晶點
4 05膠化不良
5 06黑點
6 07異物
7 08破孔
8 09結疤
9 10油污
10 11蚊蟲
11 12其他
1     4183
3     2335
8     1564
10    1004
2      807
5      649
7      468
6      439
4      434
9      120
11      13
0        6
Name: class, dtype: int64
12022


,file_path,class
0,C:/Users/n000171823/Desktop/PQ42/training/01刮傷...,0
1,C:/Users/n000171823/Desktop/PQ42/training/01刮傷...,0
2,C:/Users/n000171823/Desktop/PQ42/training/01刮傷...,0
3,C:/Users/n000171823/Desktop/PQ42/training/01刮傷...,0
4,C:/Users/n000171823/Desktop/PQ42/training/01刮傷...,0


In [5]:
#shuffle data table
from sklearn.model_selection import ShuffleSplit
from sklearn.utils import shuffle

####  刪除出現 Thumbs.db 的 row

del_list = []
for i in range(0,len(trainSet)):
    if 'Thumbs' in trainSet.loc[i,'file_path']:
        del_list.append(i)

trainSet.drop(trainSet.index[del_list], inplace=True)
print(len(trainSet))


df = shuffle(trainSet)
df.head(5)
a = df['file_path'][:1].tolist()[0]
img = cv2.imdecode(np.fromfile(a, dtype=np.uint8), cv2.IMREAD_UNCHANGED)

12010


In [6]:
#train data prepare
%matplotlib inline
import matplotlib.pyplot as plt
import cv2
#X for image data
a=0
X = []

files = df['file_path']
print(len(files))


for i in files:
    img = cv2.imdecode(np.fromfile(i, dtype=np.uint8), cv2.IMREAD_UNCHANGED)
    img = cv2.resize(img, (128,128))
    img = np.resize(img,(128,128,3))
    img_new = img.astype('float32')/255
    X.append(img_new) 
X = np.array(X)

12010


In [7]:
print(len(X))
all_index = files.index.values
print(all_index)

X_index, y_index = train_test_split( all_index, test_size=0.1, random_state=1000)
print(X_index)

12010
[ 2873  2475  2119 ...  3320  3408 11444]
[ 8934  9905 11435 ...  1061  8412  6643]


In [8]:
from keras.utils import to_categorical
y_class = np.array(df['class'])
y = to_categorical(y_class)
print(y[:5])
print(y_class[:5])

[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]]
[1 1 1 1 3]


## 模型建立

In [9]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.1, random_state=1000)

In [10]:
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers import Input
from keras.models import Model
from keras.layers import LeakyReLU
from keras.applications import InceptionV3
from keras.applications import MobileNetV2
from keras.applications import DenseNet169
from keras.applications import DenseNet201
from keras.applications import InceptionResNetV2
from keras.applications import VGG19
from keras.applications import NASNetLarge
from keras.applications import Xception
from keras.applications import NASNetMobile
import efficientnet.keras as efn
from keras import regularizers

In [11]:
img_x = 128
img_y = 128
img_channels = 3

#接上自訂輸出
temp_model = Sequential()
temp_model.add(efn.EfficientNetB7(include_top=False, pooling='avg', weights='imagenet',input_shape=(img_x, img_y, img_channels)))
temp_model.add(Dropout(0.25))
temp_model.add(Dense(1000, input_dim=1000,
                kernel_regularizer=regularizers.l2(0.01)))
# temp_model.add(BatchNormalization())
# temp_model.add(Activation('relu'))
temp_model.add(Dense(12, activation='softmax'))







Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [12]:
temp_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnet-b7 (Model)      (None, 2560)              64097680  
_________________________________________________________________
dropout_1 (Dropout)          (None, 2560)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1000)              2561000   
_________________________________________________________________
dense_2 (Dense)              (None, 12)                12012     
Total params: 66,670,692
Trainable params: 66,359,972
Non-trainable params: 310,720
_________________________________________________________________


In [13]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
# Let's train the model using Adam
model_path = os.getcwd() + '/PQ42_model_EfficientNetB7_1.h5'
temp_model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.00005),
              metrics=['accuracy'])

checkpoint = ModelCheckpoint(model_path, monitor='val_loss', save_best_only=True, verbose=1)
early_stopper = EarlyStopping(monitor='val_loss', patience=10, verbose=1)
lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=np.sqrt(0.1), cooldown=0, patience=1, min_lr=0.1e-8)

In [14]:
#model_InceptionResNet.h5: epoch = 30, Lr = 0.000001
#model_InceptionResNet2.h5: epoch = 40, Lr = 0.0001
#model_InceptionResNet_3.h5: epoch = 40,Lr = 0.00008, add new data 
#model_InceptionResNet_4.h5: epoch = 40,Lr = 0.00008, add new data but not all
#model_InceptionResNet_5.h5: epoch = 50,Lr = 0.000001, add new data, delete augmentation
#model_InceptionResNet_7.h5: epoch = 60,Lr = 0.000001, dropout = 0.85, add Allen data


import time
tStart = time.time()#計時開始


model_history =temp_model.fit(X_train, y_train,
             batch_size=8,
             epochs=60,
             validation_split=0.1,
             shuffle=True,
             verbose=1,
             callbacks=[early_stopper,checkpoint, lr_reducer])

tEnd = time.time()#計時結束
#列印結果
print ("It cost %f sec" % (tEnd - tStart))
print (tEnd - tStart)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 9728 samples, validate on 1081 samples
Epoch 1/60
9728/9728 [==============================] - 536s 55ms/step - loss: 9.5904 - acc: 0.8304 - val_loss: 4.8846 - val_acc: 0.9547

Epoch 00001: val_loss improved from inf to 4.88459, saving model to C:\Users\n000171823\Documents/PQ42_model_EfficientNetB7_1.h5
Epoch 2/60
9728/9728 [==============================] - 488s 50ms/step - loss: 2.8529 - acc: 0.9229 - val_loss: 1.3231 - val_acc: 0.9639

Epoch 00002: val_loss improved from 4.88459 to 1.32307, saving model to C:\Users\n000171823\Documents/PQ42_model_EfficientNetB7_1.h5
Epoch 3/60
9728/9728 [==============================] - 474s 49ms/step - loss: 0.8276 - acc: 0.9441 - val_loss: 0.4136 - val_acc: 0.9685

Epoch 00003: val_loss improved from 1.32307 to 0.41357, saving model to C:\Users\n000171823\Documents/PQ42_model_EfficientNetB7_1.h5
Epoch 4/60
9728/9728 [===========================

## 模型驗證

In [15]:
model_path = 'C:/Users/n000171823/Documents/PQ42_model_EfficientNetB7_1.h5'
model_best = load_model(model_path,compile=False)

In [16]:
pred = model_best.predict(X_test)
final_pred = np.array(np.argmax(pred, axis=1).tolist())

In [17]:
y_test = pd.DataFrame(y_test)
y_test_class = pd.get_dummies(y_test).idxmax(1)
print(len(y_test))
print(y_test_class[:5])

1201
0    1
1    1
2    3
3    5
4    1
dtype: int64


In [18]:
pd.crosstab(y_test_class, final_pred, rownames=['Label'], colnames=['Predict'])

Predict,1,2,3,4,5,6,7,8,9,10
Label,,,,,,,,,,
1,422,0,0,0,0,0,0,0,0,0
2,0,80,0,0,0,0,2,0,0,0
3,0,0,231,0,3,0,0,0,0,0
4,0,0,0,39,0,0,0,0,0,0
5,0,0,3,2,56,1,0,1,0,0
6,0,0,0,0,0,42,0,0,0,0
7,0,1,0,0,0,0,38,1,0,0
8,0,0,0,1,3,0,1,169,1,0
9,0,0,0,0,0,0,0,0,10,0


## 利用門檻值(thereshold)剔除未知圖像

In [23]:
Smat = model_best.predict_proba(X_test)

In [24]:
# y_test_class != final_pred
# print(type(y_test_class))
# print(type(final_pred))

y_test_class_1 = y_test_class.tolist()
final_pred_1 = final_pred.tolist()

y_mistake_index = []
for i in range(0,len(y_test_class_1)):
    if y_test_class_1[i] != final_pred_1[i]:
        y_mistake_index.append(i)
        print('true class :',y_test_class_1[i],'pred_class :',final_pred_1[i])
print(y_mistake_index)

true class : 5 pred_class : 3
true class : 5 pred_class : 3
true class : 8 pred_class : 9
true class : 8 pred_class : 4
true class : 3 pred_class : 5
true class : 3 pred_class : 5
true class : 7 pred_class : 8
true class : 8 pred_class : 5
true class : 5 pred_class : 4
true class : 5 pred_class : 4
true class : 11 pred_class : 8
true class : 8 pred_class : 5
true class : 2 pred_class : 7
true class : 7 pred_class : 2
true class : 5 pred_class : 3
true class : 8 pred_class : 5
true class : 5 pred_class : 8
true class : 3 pred_class : 5
true class : 2 pred_class : 7
true class : 5 pred_class : 6
true class : 8 pred_class : 7
[3, 237, 367, 387, 399, 447, 459, 471, 534, 555, 571, 594, 669, 800, 808, 936, 942, 1078, 1117, 1123, 1142]


In [25]:
Spmat = np.delete(Smat,y_mistake_index,axis=0)
correct_y_test_class = np.delete(list(y_test_class),y_mistake_index)

In [28]:
correct_y_test_class

array([1, 1, 3, ..., 3, 8, 3])

In [37]:
thereshold = []
for j in range(11):
    t1 = []
    for i in range(0,len(correct_y_test_class)):
        if correct_y_test_class[i] == j:
            t1.append(i)
    if len(t1) == 0:
        thereshold.append(0)
    else:
        thereshold.append(min(Spmat[t1,j]))

In [38]:
#thereshold of model_InceptionResNet_7
thereshold

[0,
 0.98692673,
 0.6674977,
 0.5074415,
 0.9888137,
 0.5484375,
 0.37657553,
 0.5610079,
 0.45735615,
 0.97635776,
 0.983247]

In [45]:
unknown = []
for j in range(1,11):
    tt = []
    for i in range(0,len(final_pred)):
        if final_pred[i] == j:
            tt.append(i)
    tt = np.array(tt)
    Smat_test_cj = Smat[tt,j]
    unknown.append(tt[Smat_test_cj < thereshold[j]].tolist())

In [46]:
unknown

[[], [], [], [534, 555], [399, 936], [], [], [571], [367], []]